## Testing notebook showing basic functionality

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from calc_utils import kml_to_shp, load_shp, normalize_shps, shp_to_land, plot_land, observations_to_circles, expand_observations, \
                       daily_score_union, daily_attibution, monthly_attribution, cummulative_attribution, daily_video, create_value_lands, plot_value_lands, transform_one_row_per_value

In [2]:
# transform plots kml to shp
kml_to_shp(source_directory='test_data/KML/', destination_directory='test_data/SHP/', original_shp_directory=None)
land_metadata = pd.read_csv('test_data/land_metadata.csv', dtype={'plot_id': str})

########## Converting 3 ##########
Converted 3.kml to 3.shp
########## Converting 18 ##########
Converted 18.kml to 18.shp
########## Converting 9 ##########
Converted 9.kml to 9.shp


Warning 6: Normalized/laundered field name: 'description' to 'descriptio'
Warning 1: The output driver does not natively support DateTime type for field timestamp. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field timestamp create as date field, though DateTime requested.
Warning 1: The output driver does not natively support DateTime type for field begin. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field begin create as date field, though DateTime requested.
Warning 1: The output driver does not natively support DateTime type for field end. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field end create as date field, though DateTime requested.
Warning 6: Normalized/laundered field name: 'altitudeMode' to 'altitudeMo'
ERROR 1: Attempt to write non-point (POLYGON) geometry to point shapefile.
ERROR 1: Unable to write feature 14 from

In [4]:
# loading shp files
shp = load_shp('test_data/SHP/')
# some preprocessing of shps
normalized_shapes = normalize_shps(shp)
plots = shp_to_land(normalized_shapes)
plots = plots.reset_index()
plots['plot_id'] = plots['index'].astype(str).str.zfill(3)
plots = plots.merge(land_metadata, on='plot_id', how='left')

subtypes = load_shp('credit_subtypes/SHP/')
platinum = subtypes['Tropical Andes']['geometry'][0]
plots, platinum_gdf = create_value_lands(plots, platinum)
# creating html map
plot_value_lands(plots, platinum_gdf, filename='test_plots_value.html')
print('You can see the map at test_plots.html')
plots.head()

Geometry type Point found in plots: 18, 3, 9
CRS found in plots: EPSG:4326: 3
Total plots processed: 3
You can see the map at test_plots.html


,index,total_area,plot_id,POD,project_biodiversity,area_certifier,geometry,value
0,18,142.034201,18,VGZ,2,130.0,"POLYGON Z ((-76.79334 0.87410 0.00000, -76.787...",platinum
1,3,64.834050,3,VGZ,2,46.5,"POLYGON Z ((-76.78710 0.88356 0.00000, -76.781...",platinum
2,9,10.096988,9,VGZ,2,10.0,"POLYGON Z ((-76.75636 0.88951 0.00000, -76.757...",platinum
3,9,10.096988,9,VGZ,2,10.0,"POLYGON Z ((-76.75475 0.89361 0.00000, -76.754...",gold


In [ ]:
# loading observations
records = pd.read_csv('test_data/observations.csv')
print('Number of test observations: ', len(records))
records.head()

In [ ]:
# from observations to circles
# crs for observations is 4326 because it is lat/lon
# crs for circles is 6262 because it is in meters
records = observations_to_circles(records, default_crs=4326, buffer_crs=6262)
records.head()

In [ ]:
# each observation is "expanded" to the 29 days before and 30 days after the observation date
obs_expanded = expand_observations(records)
obs_expanded.head()

In [ ]:
# the expanded observations are unioned to create a daily geometry for each score
# this process the venn_decomposition function, which decomposes a list of scored polygons into 
# a venn-like diagram, assigning the highest score to each section and merging the equal-score sections
daily_score = daily_score_union(obs_expanded)
daily_score.head()

In [ ]:
# just to visualize these geometries, we can pick a date and plot both scores
date = '2023-07-01'
print('Polygons for date: ', date)  
print('Displayed in separate plots for each score')
fig, ax = plt.subplots(1,2, figsize=(12,6))
daily_score.query(f'date == "{date}" and score == 0.5').plot(ax=ax[0])
daily_score.query(f'date == "{date}" and score ==  1').plot(ax=ax[1])
ax[0].set_xlim([-76.85, -76.69])
ax[0].set_ylim([0.81, 0.96])
ax[1].set_xlim([-76.85, -76.69])
ax[1].set_ylim([0.81, 0.96])
ax[0].set_title('Score 0.5')
ax[1].set_title('Score 1.0')
daily_score.query(f'date == "{date}"')

In [ ]:
# now we can calculate the daily attribution for each plot_id-score-date combination
# the area_score is the area of the intersection of the score polygon and the plot polygon multiplied by the score
attribution = daily_attibution(daily_score, plots, obs_expanded, crs=6262)
attribution.head()

In [ ]:
# now we can compute the monthly and cummulative attibution
# the monthly attribution is the sum of the area_score for each plot_id-month combination divided by 60
# calc_index is the plot_id-month combination
attr_month = monthly_attribution(attribution)
attr_cumm = cummulative_attribution(attr_month, cutdays = 30, start_date=None)
attr_month = transform_one_row_per_value(attr_month, 'month')
attr_cumm = transform_one_row_per_value(attr_cumm, 'cumm')
attr_month.head()

In [ ]:
attr_cumm.head()


In [ ]:
# finally for visualization we can create a video called raindrops.mp4
daily_video(daily_score, plots, first_date=None)